In this notebook, the timed scenario runs and we can get back the predictions. 
It is needed still to analyze the results

In [ ]:
import tqdm as notebook_tqdm
import pandas as pd
from datetime import datetime

# from recpack.pipelines import PipelineBuilder
from PipelineBuilder_modified import * 
from recpack.scenarios import WeakGeneralization, Timed
from recpack.preprocessing.preprocessors import DataFramePreprocessor
from recpack.preprocessing.filters import MinItemsPerUser, MinUsersPerItem

In [ ]:
transactions_path = '../../00 - Data/transactions/transactions_train.csv'
transactions = pd.read_csv(transactions_path)

In [ ]:
#1:  Data collection
transactions_path = '../../00 - Data/transactions/transactions_train.csv'
transactions = pd.read_csv(transactions_path)
print("Original data has size of : " + str(len(transactions)))

#transform datetime to unix epoch
transactions['timestamp'] = pd.to_datetime(transactions['t_dat']).astype(int) / 10**9
transactions.drop(columns=['t_dat'], inplace=True)

articles_path = '../../00 - Data/articles/articles.csv'
articles_df = pd.read_csv(articles_path)


In [ ]:
transactions.t_dat.max()

In [ ]:
#2: Data preprocessing

#        item1    item2   item3
#usr1      x                x
#usr2       x       x
proc = DataFramePreprocessor(item_ix='article_id', user_ix='customer_id', timestamp_ix='timestamp')
# #every user has at least 2 items bought
proc.add_filter(MinUsersPerItem(5, item_ix='article_id', user_ix='customer_id'))
# #every item is bought at least twice
proc.add_filter(MinItemsPerUser(5, item_ix='article_id', user_ix='customer_id'))

interaction_matrix = proc.process(transactions)

In [ ]:
from datetime import datetime
#As i really dont have validation data (it is hidden by kaggle, i set both values to same date). 'delta_in' before 't' will be used for training and 'delta_out' weeks after 't' will be used for testing
t = datetime(2020, 9, 15).timestamp()
t_validation = datetime(2020, 9, 14).timestamp()
#maybe 9?
delta_in = 1 * 604800
# 1 semana = 604800
delta_out = 604800

#3 : Create scenario
scenario = Timed(t, t_validation=t_validation, validation=True, delta_in = delta_in, delta_out = delta_out, seed =1)
# scenario = Timed() 
scenario.split(interaction_matrix)

In [ ]:
#4 : Create the builder object
builder = PipelineBuilder()
builder.set_data_from_scenario(scenario)

#adds algorithms to use later on. Baseline algorithim, just recommends popular stuff
# builder.add_algorithm('Popularity') 
# builder.add_algorithm('ItemKNN', grid={
#     'K': [100, 200, 500],
#     'similarity': ['cosine', 'conditional_probability'],
# })
builder.add_algorithm('KUNN')

#Set the metric for optimisation of parameters in algorithms. What is NDCGK ??
builder.set_optimisation_metric('NDCGK', K=10)

#adds metric for evaluation
#NDCGK = Normalized Discounted Cumulative Gain at K
builder.add_metric('NDCGK', K=[10, 20, 50])
builder.add_metric('CoverageK', K=[10, 20])

In [ ]:
# #5 : Create and run the pipeline
# pipeline = builder.build()
# pipeline.run()
# # x_preds = pipeline.run2()

# #6 : Get results
# pipeline.get_metrics()
# # pipeline.optimisation_results
# #pipeline.saveResults()

from aux_functions import *
from PipelineBuilder_modified import * 

pipeline = builder.build()
csr = pipeline.run2()
user_rec = UserRecommendations.fill_user_rec(csr)

In [ ]:
#ITEM BEING RECOMMENDED
# user_rec.get_rec_user(6)[0][0]
article_ids_array = user_rec.get_article_ids()
decoded_items = []
item_id_mapping = proc.item_id_mapping.set_index(interaction_matrix.ITEM_IX)[proc.item_ix].to_dict()

for article_id in article_ids_array:

    if article_id in item_id_mapping:
        decoded_item = item_id_mapping[article_id]
        decoded_items.append(decoded_item)

article_counts = pd.Series(decoded_items).value_counts().to_dict()

# Filter 'articles_df' based on article IDs present in 'decoded_items'
recommended_articles = articles_df[articles_df['article_id'].isin(decoded_items)].copy()

# Count occurrences of each article ID in 'decoded_items'
article_counts = pd.Series(decoded_items).value_counts().to_dict()

# Map counts back to 'filtered_articles_df' using .loc to avoid SettingWithCopyWarning
recommended_articles.loc[:, 'number_of_repeats'] = recommended_articles['article_id'].map(article_counts).fillna(0).astype(int)

#recommended_articles contains a dataframe with all my new recommendations
recommended_articles.to_csv("recommended_articles_pop.csv")

In [ ]:
recommended_articles